In [1]:
!pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!pip install bitsandbytes==0.41.0


# Cài transformers, trl, peft, accelerate
!pip install transformers==4.36.2 --force-reinstall --no-cache-dir
!pip install trl==0.7.1
!pip install peft==0.7.1
!pip install accelerate==0.26.1
!pip install datasets


Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/126.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/126.8 kB ? eta -:--:--
     ----------- ------------------------- 41.0/126.8 kB 393.8 kB/s eta 0:00:01
     ----------------- ------------------- 61.4/126.8 kB 469.7 kB/s eta 0:00:01
     ------------------------------- ---- 112.6/126.8 kB 595.3 kB/s eta 0:00:01
     ------------------------------------ 126.8/126.8 kB 619.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     -------------------------- ------------- 41.0/60.8 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/41.5 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 666.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip uninstall -y triton
!pip install triton==2.1.0

Found existing installation: triton 3.3.0
Uninstalling triton-3.3.0:
  Successfully uninstalled triton-3.3.0
ERROR: Could not find a version that satisfies the requirement triton==2.1.0 (from versions: 2.2.0, 2.3.0, 2.3.1, 3.0.0, 3.1.0, 3.2.0, 3.3.0)
ERROR: No matching distribution found for triton==2.1.0


In [2]:
import torch, transformers, trl, peft

print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("trl:", trl.__version__)
print("peft:", peft.__version__)
print("CUDA:", torch.cuda.is_available())


d:\AI17C\Capstone AI17C\lora_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch: 2.1.0+cu118
transformers: 4.36.2
trl: 0.7.1
peft: 0.7.1
CUDA: True


In [8]:
import os
import gc
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    HfArgumentParser, TrainingArguments, pipeline, logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [4]:
import json

input_file = 'medquad_dataset.jsonl'
output_file = 'medquad_llama_chat_format.jsonl'

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        item = json.loads(line)
        question = item['question'].strip().replace('\n', ' ')
        answer = item['answer'].strip().replace('\n', ' ')
        llama_chat_format = f"<s>[INST] {question} [/INST] {answer} </s>"
        outfile.write(json.dumps({"text": llama_chat_format}) + '\n')


In [ ]:
import json
import random

# Đọc dữ liệu gốc
with open('medquad_llama_chat_format.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

# Xáo trộn dữ liệu
random.seed(42)
random.shuffle(data)

# Chia theo tỉ lệ 80% train, 10% valid, 10% test
n_total = len(data)
n_train = int(0.9 * n_total)

train_data = data[:n_train]
valid_data = data[n_train:]

# Ghi ra file
with open('train.jsonl', 'w') as f:
    for item in train_data:
        f.write(json.dumps(item) + '\n')

with open('valid.jsonl', 'w') as f:
    for item in valid_data:
        f.write(json.dumps(item) + '\n')


In [9]:
base_model_name = 'NousResearch/Llama-2-7b-chat-hf'
finetune_model_name='my_model_finetune_llama2_7b'

output_dir = './results'

no_of_epochs = 1

# No change params
use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "int4", "nf4", False # To quantization
lora_r, lora_alpha, lora_dropout = 64, 16, 0.1
fp16, bf16 =  False, False
per_device_train_batch_size, per_device_eval_batch_size = 4, 4
gradient_accumulation_steps, gradient_checkpointing, max_grad_norm = 1, True, 0.3
learning_rate, weight_decay, optim = 2e-4, 0.001, "paged_adamw_32bit"
lr_scheduler_type, max_steps, warmup_ratio = "cosine", -1, 0.03
group_by_length, save_steps, logging_steps = True, 0, 25
max_seq_length, packing, device_map = None, False, {"": 0}


In [10]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

/mnt/d/AI17C/Capstone AI17C/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):
No module named 'triton.ops'

In [ ]:
# Cấu hình LoRA
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs= no_of_epochs, 
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)


NameError: name 'num_train_epochs' is not defined

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=load_dataset('json', data_files='train.jsonl', split='train'),
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()